In [ ]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [ ]:
train_dir = "../input/chest-xray-pneumonia/chest_xray/train"
test_dir = "../input/chest-xray-pneumonia/chest_xray/test"
val_dir = "../input/chest-xray-pneumonia/chest_xray/val"

In [ ]:
num_pneumonia = len(os.listdir(os.path.join(train_dir,'PNEUMONIA')))
num_normal = len(os.listdir(os.path.join(train_dir, 'NORMAL')))


In [ ]:
print(f"PNEUMONIA={num_pneumonia}")
print(f"NORMAL={num_normal}")

In [ ]:
print(f"PNEUMONIA={len(os.listdir(os.path.join(test_dir, 'PNEUMONIA')))}")
print(f"NORMAL={len(os.listdir(os.path.join(test_dir, 'NORMAL')))}")

In [ ]:
print(f"PNEUMONIA={len(os.listdir(os.path.join(val_dir, 'PNEUMONIA')))}")
print(f"NORMAL={len(os.listdir(os.path.join(val_dir, 'NORMAL')))}")

In [ ]:
pneumonia = os.listdir("../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA")
pneumonia_dir = "../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA"

In [ ]:
NORMAL = os.listdir("../input/chest-xray-pneumonia/chest_xray/train/NORMAL")
NORMAL_dir = "../input/chest-xray-pneumonia/chest_xray/train/NORMAL"

In [ ]:
plt.figure(figsize=(15, 5))

for i in range(12):
    plt.subplot(4, 3, i + 1)
    img = plt.imread(os.path.join(NORMAL_dir, NORMAL[i]))
    plt.title("NORMAL")
    plt.imshow(img, cmap='gray')
    plt.axis('on')
    

plt.tight_layout()

In [ ]:
plt.figure(figsize=(15, 5))

for i in range(12):
    plt.subplot(4, 3, i + 1)
    img = plt.imread(os.path.join(pneumonia_dir, pneumonia[i]))
    plt.title("PNEUMONIA")
    plt.imshow(img, cmap='gray')
    plt.axis('on')
    

plt.tight_layout()

In [ ]:
import glob

pneumonia_train = glob.glob(train_dir+"/PNEUMONIA/*.jpeg")
normal_train = glob.glob(train_dir+"/NORMAL/*.jpeg")

In [ ]:
data = pd.DataFrame(np.concatenate([[0]*len(normal_train) ,
                                    [1]*len(pneumonia_train)]),columns=["class"])

In [ ]:
data.head(4000)

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(data['class'],data=data,palette='rocket',color='red')
plt.title('PNEUMONIA vs NORMAL')
plt.show()

In [ ]:
img_Datagen = ImageDataGenerator(
        rescale = 1/255,
        featurewise_center=True,
        featurewise_std_normalization=True,
        shear_range=10,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=[0.5,2.0],
        width_shift_range = 0.2,
        rotation_range=20,
        fill_mode = 'nearest'
)
val_Datagen = ImageDataGenerator(
        rescale = 1/255
)

In [ ]:
train = img_Datagen.flow_from_directory(train_dir,
                                       batch_size=8,
                                       class_mode='binary'
                                       )

In [ ]:

validation = val_Datagen.flow_from_directory(val_dir,
                                              batch_size=8,
                                              class_mode='binary',
                                              )

In [ ]:
test = val_Datagen.flow_from_directory(test_dir,
                                       batch_size=8,
                                       class_mode='binary',
                                      )

In [ ]:
vgg_model = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top = False,
   )
    
x = vgg_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
predictions = tf.keras.layers.Dense(1,activation='sigmoid')(x)
model = tf.keras.Model(inputs=vgg_model.input, outputs=predictions)

'TF_GPU_ALLOCATOR=cuda_malloc_async'

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models
    

In [ ]:
import segmentation_models as sm

sm.set_framework('tf.keras')

sm.framework()

In [ ]:
vgg_model = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top = False,
)

from tensorflow.keras.applications import EfficientNetB0
modeleff = EfficientNetB0(weights='imagenet',include_top = False,)

for layer in modeleff.layers:
    layer.trainable=False
    
x = modeleff.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64,activation='relu')(x)
# output layer
predictions = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs=modeleff.input, outputs=predictions)

In [ ]:
# to avoid overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',patience=8)

In [ ]:
# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['Precision'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train,epochs=30, 
                    validation_data=validation,
                     steps_per_epoch=50,
                    callbacks=[early_stopping,lr],
                    verbose=True,
                    batch_size=8)
                

In [ ]:
score = model.evaluate(train)

print("Train Loss is: ", score[0])
print("Train Accuracy is: ", score[1])

In [ ]:
score = model.evaluate(test)

print("Test Loss is: ", score[0])
print("Test Accuracy is: ", score[1])

In [ ]:
plt.figure(figsize=(12, 8))
plt.title('EVALUATION OF VGG16')

plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

In [ ]:
prediction1=model.predict(test)

In [ ]:
prediction1[:10]

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(history.history['precision'], label='Precision')
plt.plot(history.history['val_precision'], label='Val_Precision')
plt.legend()
plt.title('Precision Evolution')